In [1]:
from dask.distributed import Client
import numpy as np
from scipy.interpolate import griddata  
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import intake
import dask
import multiprocessing
import xarray as xr
import sys
ncpu = multiprocessing.cpu_count()
threads = 6
nworker = ncpu//threads
print(f'Number of CPUs: {ncpu}, number of threads: {threads}, number of workers: {nworker}')

Number of CPUs: 256, number of threads: 6, number of workers: 42


In [3]:
client = Client(processes=True, threads_per_worker=threads, n_workers=nworker, memory_limit='32GB')
client


2023-05-04 13:39:43,969 - distributed.diskutils - INFO - Found stale lock file and directory '/home/b/b381258/notebooks/dask-worker-space/worker-kz5yzq0_', purging
2023-05-04 13:39:43,977 - distributed.diskutils - INFO - Found stale lock file and directory '/home/b/b381258/notebooks/dask-worker-space/worker-be3n_wjp', purging
2023-05-04 13:39:43,983 - distributed.diskutils - INFO - Found stale lock file and directory '/home/b/b381258/notebooks/dask-worker-space/worker-uqty8x7l', purging
2023-05-04 13:39:43,987 - distributed.diskutils - INFO - Found stale lock file and directory '/home/b/b381258/notebooks/dask-worker-space/worker-kkn8b6v7', purging
2023-05-04 13:39:43,991 - distributed.diskutils - INFO - Found stale lock file and directory '/home/b/b381258/notebooks/dask-worker-space/worker-4pd6es52', purging
2023-05-04 13:39:43,996 - distributed.diskutils - INFO - Found stale lock file and directory '/home/b/b381258/notebooks/dask-worker-space/worker-vgcen2ia', purging
2023-05-04 13:39

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 42
Total threads: 252,Total memory: 1.22 TiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:37245,Workers: 42
Dashboard: http://127.0.0.1:8787/status,Total threads: 252
Started: Just now,Total memory: 1.22 TiB
Comm: tcp://127.0.0.1:38521,Total threads: 6
Dashboard: http://127.0.0.1:37477/status,Memory: 29.80 GiB
Nanny: tcp://127.0.0.1:39997,


In [5]:
!ls /work/ik1017/CMIP6/data/CMIP6/CMIP/CNRM-CERFACS/CNRM-CM6-1/historical | wc

     30      30     291


In [31]:
!ls /work/ik1017/CMIP6/data/CMIP6/CMIP/CNRM-CERFACS/CNRM-CM6-1/historical/r26i1p1f2/Eday/ua/gr/v20191004


ua_Eday_CNRM-CM6-1_historical_r26i1p1f2_gr_19350101-19391231.nc


In [25]:
!ls /work/ik1017/CMIP6/data/CMIP6/CMIP/CNRM-CERFACS/CNRM-CM6-1/historical/r27i1p1f2/Eday

mrsll  ta500  zg


In [47]:
!ls -lrt /work/ik1017/CMIP6/data/CMIP6/CMIP/INM/INM-CM5-0/historical/

total 40
drwxrwsr-x 14 k204210 esgf 4096 Oct 12  2019 r10i1p1f1
drwxrwsr-x 14 k204210 esgf 4096 Oct 13  2019 r2i1p1f1
drwxrwsr-x 14 k204210 esgf 4096 Oct 15  2019 r1i1p1f1
drwxrwsr-x 14 k204210 esgf 4096 Oct 16  2019 r5i1p1f1
drwxrwsr-x 14 k204210 esgf 4096 Oct 17  2019 r6i1p1f1
drwxrwsr-x 14 k204210 esgf 4096 Oct 17  2019 r7i1p1f1
drwxrwsr-x 14 k204210 esgf 4096 Oct 19  2019 r8i1p1f1
drwxrwsr-x 14 k204210 esgf 4096 Oct 20  2019 r9i1p1f1
drwxrwsr-x 13 k204210 esgf 4096 Nov 12  2019 r4i1p1f1
drwxrwsr-x 13 k204210 esgf 4096 Nov 12  2019 r3i1p1f1


In [45]:
!ls -lrt /work/ik1017/CMIP6/data/CMIP6/CMIP/INM/INM-CM5-0/historical/r1i1p1f1/day/ua/gr1/v20190610

total 29890248
-r--r--r-- 1 k204210 esgf 927077771 Oct 15  2019 ua_day_INM-CM5-0_historical_r1i1p1f1_gr1_18500101-18541231.nc
-r--r--r-- 1 k204210 esgf 927480663 Oct 15  2019 ua_day_INM-CM5-0_historical_r1i1p1f1_gr1_18550101-18591231.nc
-r--r--r-- 1 k204210 esgf 927368329 Oct 15  2019 ua_day_INM-CM5-0_historical_r1i1p1f1_gr1_18600101-18641231.nc
-r--r--r-- 1 k204210 esgf 927791237 Oct 15  2019 ua_day_INM-CM5-0_historical_r1i1p1f1_gr1_18650101-18691231.nc
-r--r--r-- 1 k204210 esgf 928465814 Oct 15  2019 ua_day_INM-CM5-0_historical_r1i1p1f1_gr1_18700101-18741231.nc
-r--r--r-- 1 k204210 esgf 928084670 Oct 15  2019 ua_day_INM-CM5-0_historical_r1i1p1f1_gr1_18750101-18791231.nc
-r--r--r-- 1 k204210 esgf 927362101 Oct 15  2019 ua_day_INM-CM5-0_historical_r1i1p1f1_gr1_18800101-18841231.nc
-r--r--r-- 1 k204210 esgf 927557418 Oct 15  2019 ua_day_INM-CM5-0_historical_r1i1p1f1_gr1_18850101-18891231.nc
-r--r--r-- 1 k204210 esgf 928131166 Oct 15  2019 ua_day_INM-CM5-0_historical_r1i1p1f1_gr1_1890010

In [12]:
!ls -sh /work/ik1017/CMIP6/data/CMIP6/CMIP/IPSL/IPSL-CM6A-LR/historical/r28i1p1f1/

total 100K
4.0K 3hr      4.0K CF3hr   4.0K EdayZ  4.0K LImon  4.0K Ofx
4.0K 6hrLev   4.0K day	   4.0K Emon   4.0K Lmon   4.0K Omon
4.0K AERmon   4.0K E3hr    4.0K EmonZ  4.0K Oclim  4.0K Oyr
4.0K AERmonZ  4.0K E3hrPt  4.0K Eyr    4.0K Oday   4.0K SIday
4.0K Amon     4.0K Eday    4.0K fx     4.0K Odec   4.0K SImon


In [33]:
!ls -lrt /pool/data/Catalogs/

total 8
lrwxrwxrwx  1 k204210 bm0021   42 Apr  8  2022 dkrz_cmip6_disk.json -> /work/ik1017/Catalogs/dkrz_cmip6_disk.json
lrwxrwxrwx  1 k204210 bm0021   44 Apr 12  2022 dkrz_palmod2_disk.json -> /work/ik1017/Catalogs/dkrz_palmod2_disk.json
lrwxrwxrwx  1 k204210 bm0021   42 Apr 12  2022 dkrz_cmip5_disk.json -> /work/ik1017/Catalogs/dkrz_cmip5_disk.json
lrwxrwxrwx  1 k204210 bm0021   45 Apr 13  2022 dkrz_nextgems_disk.json -> /work/ik1017/Catalogs/dkrz_nextgems_disk.json
lrwxrwxrwx  1 k204210 bm0021   51 Apr 13  2022 dkrz_dyamond-winter_disk.json -> /work/ik1017/Catalogs/dkrz_dyamond-winter_disk.json
drwxr-xr-x  2 k204210 bm0021 4096 Apr 14  2022 Templates
lrwxrwxrwx  1 k204210 bm0021   43 Apr 14  2022 dkrz_cordex_disk.json -> /work/ik1017/Catalogs/dkrz_cordex_disk.json
lrwxrwxrwx  1 k204210 bm0021   43 May  9  2022 dkrz_cmip6_cloud.json -> /work/ik1017/Catalogs/dkrz_cmip6_cloud.json
lrwxrwxrwx. 1 k204210 bm0021   39 Jun  9  2022 dkrz_catalog.yaml -> /work/ik1017/Catalogs/dkrz_catalog.ya

In [2]:
root_path = '/work/ik1017/Catalogs/'
#root_path = '/pool/data/Catalogs/Candidates/'
#root_path = '/pool/data/Catalogs/'
#col_url = f"{root_path}mistral-cmip6.json"
col_url = f"{root_path}dkrz_cmip-data-pool_disk_netcdf.json"
col_url = f"{root_path}dkrz_cmip6_disk.json"
#col_url = f"{root_path}dkrz_cmip6_disk.json"
col = intake.open_esm_datastore(col_url)
col

/sw/spack-levante/mambaforge-4.11.0-0-Linux-x86_64-sobz6z/lib/python3.9/site-packages/intake_esm/utils.py:96: DtypeWarning: Columns (21,22,23) have mixed types. Specify dtype option on import or set low_memory=False.
  return pd.read_csv(catalog_path, **csv_kwargs), catalog_path


,unique
activity_id,20
institution_id,39
source_id,103
experiment_id,214
member_id,479
table_id,39
variable_id,1164
grid_label,12
dcpp_init_year,63
version,753


In [13]:
query = dict(experiment_id='amip', table_id='day', 
             variable_id=['ua','zg'])
cat = col.search(**query)
cat.df

,activity_id,institution_id,source_id,experiment_id,member_id,table_id,variable_id,grid_label,dcpp_init_year,version,...,frequency,time_reduction,long_name,units,realm,level_type,time_min,time_max,format,uri
0,CMIP,AS-RCEC,TaiESM1,amip,r1i1p1f1,day,ua,gn,NaN,v20200817,...,day,mean,Eastward Wind,m s-1,atmos,NaN,19790101.0,19791231,netcdf,/work/ik1017/CMIP6/data/CMIP6/CMIP/AS-RCEC/Tai...
1,CMIP,AS-RCEC,TaiESM1,amip,r1i1p1f1,day,ua,gn,NaN,v20200817,...,day,mean,Eastward Wind,m s-1,atmos,NaN,19800101.0,19891231,netcdf,/work/ik1017/CMIP6/data/CMIP6/CMIP/AS-RCEC/Tai...
2,CMIP,AS-RCEC,TaiESM1,amip,r1i1p1f1,day,ua,gn,NaN,v20200817,...,day,mean,Eastward Wind,m s-1,atmos,NaN,19900101.0,19991231,netcdf,/work/ik1017/CMIP6/data/CMIP6/CMIP/AS-RCEC/Tai...
3,CMIP,AS-RCEC,TaiESM1,amip,r1i1p1f1,day,ua,gn,NaN,v20200817,...,day,mean,Eastward Wind,m s-1,atmos,NaN,20100101.0,20101231,netcdf,/work/ik1017/CMIP6/data/CMIP6/CMIP/AS-RCEC/Tai...
4,CMIP,AS-RCEC,TaiESM1,amip,r1i1p1f1,day,ua,gn,NaN,v20200817,...,day,mean,Eastward Wind,m s-1,atmos,NaN,20110101.0,20141231,netcdf,/work/ik1017/CMIP6/data/CMIP6/CMIP/AS-RCEC/Tai...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2134,CMIP,SNU,SAM0-UNICON,amip,r1i1p1f1,day,ua,gn,NaN,v20190323,...,day,mean,Eastward Wind,m s-1,atmos,NaN,20100101,20101231.0,netcdf,/work/ik1017/CMIP6/data/CMIP6/CMIP/SNU/SAM0-UN...
2135,CMIP,SNU,SAM0-UNICON,amip,r1i1p1f1,day,ua,gn,NaN,v20190323,...,day,mean,Eastward Wind,m s-1,atmos,NaN,20110101,20111231.0,netcdf,/work/ik1017/CMIP6/data/CMIP6/CMIP/SNU/SAM0-UN...
2136,CMIP,SNU,SAM0-UNICON,amip,r1i1p1f1,day,ua,gn,NaN,v20190323,...,day,mean,Eastward Wind,m s-1,atmos,NaN,20120101,20121231.0,netcdf,/work/ik1017/CMIP6/data/CMIP6/CMIP/SNU/SAM0-UN...
2137,CMIP,SNU,SAM0-UNICON,amip,r1i1p1f1,day,ua,gn,NaN,v20190323,...,day,mean,Eastward Wind,m s-1,atmos,NaN,20130101,20131231.0,netcdf,/work/ik1017/CMIP6/data/CMIP6/CMIP/SNU/SAM0-UN...


In [14]:
model_ls = np.unique(cat.df['source_id'])
model_ls

array(['ACCESS-CM2', 'ACCESS-ESM1-5', 'BCC-CSM2-MR', 'CAMS-CSM1-0',
       'CESM2', 'CESM2-FV2', 'CESM2-WACCM', 'CESM2-WACCM-FV2',
       'CMCC-CM2-SR5', 'CNRM-CM6-1', 'CNRM-CM6-1-HR', 'CNRM-ESM2-1',
       'CanESM5', 'EC-Earth3', 'EC-Earth3-AerChem', 'FGOALS-f3-L',
       'FGOALS-g3', 'GFDL-CM4', 'HadGEM3-GC31-LL', 'HadGEM3-GC31-MM',
       'INM-CM4-8', 'INM-CM5-0', 'IPSL-CM6A-LR', 'MIROC-ES2L', 'MIROC6',
       'MPI-ESM-1-2-HAM', 'MPI-ESM1-2-HR', 'MPI-ESM1-2-LR', 'MRI-ESM2-0',
       'NorESM2-LM', 'SAM0-UNICON', 'TaiESM1'], dtype=object)

In [3]:
query = dict(experiment_id='amip', table_id='EdayZ', 
             variable_id='utendogw')#, source_id='INM-CM5-0')
cat = col.search(**query)
model_ls_ogwd = np.unique(cat.df['source_id'])
model_ls_ogwd

array(['CESM2', 'CESM2-FV2', 'CESM2-WACCM', 'CESM2-WACCM-FV2', 'CanESM5',
       'GFDL-CM4', 'HadGEM3-GC31-LL', 'HadGEM3-GC31-MM', 'INM-CM4-8',
       'INM-CM5-0', 'IPSL-CM6A-LR', 'MIROC6', 'MRI-ESM2-0'], dtype=object)

In [23]:
np.isin(model_ls_ogwd, model_ls)

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True])

In [ ]:
models = ['INM.INM-CM5-0']#['INM.INM-CM5-0','IPSL.IPSL-CM6A-LR']#,'MPI-ESM1-2-HR']#'MIROC6','UKESM1-0-LL',['CanESM5', 'MPI-ESM1-2-LR','CNRM-CM6-1']#'CESM2-WACCM']
query = dict(experiment_id='amip', table_id='day', 
             variable_id=['ua','zg'], source_id=models)
cat = col.search(**query)
cat.df

In [31]:
models = ['INM.INM-CM5-0']#['INM.INM-CM5-0','IPSL.IPSL-CM6A-LR']#,'MPI-ESM1-2-HR']#'MIROC6','UKESM1-0-LL',['CanESM5', 'MPI-ESM1-2-LR','CNRM-CM6-1']#'CESM2-WACCM']
query = dict(experiment_id='historical', table_id='Eday', 
             variable_id='ua10', source_id=models)
cat = col.search(**query)
cat.df

/sw/spack-levante/mambaforge-4.11.0-0-Linux-x86_64-sobz6z/lib/python3.9/site-packages/intake_esm/search.py:106: UserWarning: Query returned zero results.
  warn(message)


,activity_id,institution_id,source_id,experiment_id,member_id,table_id,variable_id,grid_label,dcpp_init_year,version,...,frequency,time_reduction,long_name,units,realm,level_type,time_min,time_max,format,uri


In [23]:
cat.df.iloc[0]

activity_id                                                    CMIP
institution_id                                                MIROC
source_id                                                    MIROC6
experiment_id                                            historical
member_id                                                 r10i1p1f1
table_id                                                        day
variable_id                                                      ua
grid_label                                                       gn
dcpp_init_year                                                  NaN
version                                                   v20191016
time_range                                        18500101-18501231
path              /work/ik1017/CMIP6/data/CMIP6/CMIP/MIROC/MIROC...
opendap_url       http://esgf3.dkrz.de/thredds/dodsC/cmip6/CMIP/...
project                                                       CMIP6
simulation_id                                   

In [14]:
cat.df['member_id'].unique()

array(['r10i1p1f1', 'r1i1p1f1', 'r2i1p1f1', 'r3i1p1f1', 'r4i1p1f1',
       'r5i1p1f1', 'r6i1p1f1', 'r7i1p1f1', 'r8i1p1f1', 'r9i1p1f1'],
      dtype=object)

In [21]:
with dask.config.set(**{'array.slicing.split_large_chunks': True}):
    dset_dict = cat.to_dataset_dict(cdf_kwargs={'chunks': {'time': 365, 'plev': 1}},)


--> The keys in the returned dictionary of datasets are constructed as follows:
	'activity_id.source_id.experiment_id.table_id.grid_label'


/sw/spack-levante/mambaforge-4.11.0-0-Linux-x86_64-sobz6z/lib/python3.9/site-packages/xarray/conventions.py:516: SerializationWarning: variable 'utendogw' has multiple fill values {1e+20, 1e+20}, decoding all values to NaN.
  new_vars[k] = decode_cf_variable(
/sw/spack-levante/mambaforge-4.11.0-0-Linux-x86_64-sobz6z/lib/python3.9/site-packages/xarray/conventions.py:516: SerializationWarning: variable 'utendogw' has multiple fill values {1e+20, 1e+20}, decoding all values to NaN.
  new_vars[k] = decode_cf_variable(
/sw/spack-levante/mambaforge-4.11.0-0-Linux-x86_64-sobz6z/lib/python3.9/site-packages/xarray/conventions.py:516: SerializationWarning: variable 'utendogw' has multiple fill values {1e+20, 1e+20}, decoding all values to NaN.
  new_vars[k] = decode_cf_variable(
/sw/spack-levante/mambaforge-4.11.0-0-Linux-x86_64-sobz6z/lib/python3.9/site-packages/xarray/conventions.py:516: SerializationWarning: variable 'utendogw' has multiple fill values {1e+20, 1e+20}, decoding all values to N

/sw/spack-levante/mambaforge-4.11.0-0-Linux-x86_64-sobz6z/lib/python3.9/site-packages/xarray/conventions.py:516: SerializationWarning: variable 'utendogw' has multiple fill values {1e+20, 1e+20}, decoding all values to NaN.
  new_vars[k] = decode_cf_variable(
/sw/spack-levante/mambaforge-4.11.0-0-Linux-x86_64-sobz6z/lib/python3.9/site-packages/dask/array/core.py:4697: PerformanceWarning: Increasing number of chunks by factor of 39
  result = blockwise(
/sw/spack-levante/mambaforge-4.11.0-0-Linux-x86_64-sobz6z/lib/python3.9/site-packages/xarray/conventions.py:516: SerializationWarning: variable 'utendogw' has multiple fill values {1e+20, 1e+20}, decoding all values to NaN.
  new_vars[k] = decode_cf_variable(
/sw/spack-levante/mambaforge-4.11.0-0-Linux-x86_64-sobz6z/lib/python3.9/site-packages/dask/array/core.py:4697: PerformanceWarning: Increasing number of chunks by factor of 39
  result = blockwise(
/sw/spack-levante/mambaforge-4.11.0-0-Linux-x86_64-sobz6z/lib/python3.9/site-packages/

AggregationError: 
        Failed to join/concatenate datasets in group with key=CMIP.CESM2-WACCM-FV2.amip.EdayZ.gn along a new dimension `member_id`.

        *** Arguments passed to xarray.concat() ***:

        - objs: a list of 3 datasets
        - dim: <xarray.DataArray 'member_id' (member_id: 3)>
array(['r1i1p1f1', 'r2i1p1f1', 'r3i1p1f1'], dtype='<U8')
Dimensions without coordinates: member_id
        - data_vars: ['utendogw']
        - and kwargs: {'compat': 'override', 'coords': 'minimal'}

        ********************************************
        

In [31]:
k

'CMIP.CESM2.amip.EdayZ.gn'

In [ ]:
user = 'b381258'
outpath=f'/work/bd1022/{user}/'
sel_var = 'utendogw'

for model in model_ls_ogwd:
    query = dict(experiment_id='amip', table_id='EdayZ', 
                 variable_id=sel_var, source_id=model)
    cat = col.search(**query)
    with dask.config.set(**{'array.slicing.split_large_chunks': True}):
        dset_dict = cat.to_dataset_dict(cdf_kwargs={'chunks': {'time': 365, 'plev': 1}},)
        
        for k, ds in dset_dict.items():
            ds_sel = ds.copy()
            """ ds_sel = ds['ua'].sel(
                plev = [1000],
                #time = slice(None,'2100-12-31'),
                lat = 60,
                method = 'nearest'
            )
            """
            ds_sel_winter = ds_sel[[sel_var]].sel(time = np.isin(ds_sel.time.dt.month, [11,12,1,2,3]))
            output = f'{outpath}CMIP6/{k}_{sel_var}.nc'
            print(output)




            #temp = ds_sel.sel(time = np.isin(ds_sel.time.dt.month, [11,12,1,2,3])).sel(lat = 60, method = 'nearest').mean(['lon','time'])#.load()



            #sys.exit()


            ds_sel_winter.to_netcdf(output)


--> The keys in the returned dictionary of datasets are constructed as follows:
	'activity_id.source_id.experiment_id.table_id.grid_label'


/sw/spack-levante/mambaforge-4.11.0-0-Linux-x86_64-sobz6z/lib/python3.9/site-packages/xarray/conventions.py:516: SerializationWarning: variable 'utendogw' has multiple fill values {1e+20, 1e+20}, decoding all values to NaN.
  new_vars[k] = decode_cf_variable(
/sw/spack-levante/mambaforge-4.11.0-0-Linux-x86_64-sobz6z/lib/python3.9/site-packages/xarray/conventions.py:516: SerializationWarning: variable 'utendogw' has multiple fill values {1e+20, 1e+20}, decoding all values to NaN.
  new_vars[k] = decode_cf_variable(
/sw/spack-levante/mambaforge-4.11.0-0-Linux-x86_64-sobz6z/lib/python3.9/site-packages/xarray/conventions.py:516: SerializationWarning: variable 'utendogw' has multiple fill values {1e+20, 1e+20}, decoding all values to NaN.
  new_vars[k] = decode_cf_variable(
/sw/spack-levante/mambaforge-4.11.0-0-Linux-x86_64-sobz6z/lib/python3.9/site-packages/xarray/conventions.py:516: SerializationWarning: variable 'utendogw' has multiple fill values {1e+20, 1e+20}, decoding all values to N

/work/bd1022/b381258/CMIP6/CMIP.CESM2.amip.EdayZ.gn_utendogw.nc


In [28]:
user = 'b381258'
outpath=f'/work/bd1022/{user}/'

for k, ds in dset_dict.items():
    ds_sel = ds['ua'].sel(
        plev = [1000],
        #time = slice(None,'2100-12-31'),
        lat = 60,
        method = 'nearest'
    )
    ds_sel_winter = ds_sel.sel(time = np.isin(ds_sel.time.dt.month, [11,12,1,2,3])).mean(['lon'])
    output = f'{outpath}CMIP6/{k}_ua@10hPa.nc'
    print(output)
    

    
    
    #temp = ds_sel.sel(time = np.isin(ds_sel.time.dt.month, [11,12,1,2,3])).sel(lat = 60, method = 'nearest').mean(['lon','time'])#.load()



    #sys.exit()

    
    ds_sel_winter.to_netcdf(output)
    

/work/bd1022/b381258/CMIP6/CMIP.MIROC6.historical.day.gn_ua@10hPa.nc
